In [1]:
import pandas as pd
import html
import re
from rank_bm25 import BM25Okapi

In [2]:
df = pd.read_csv('./data/formatted_dataset.csv', nrows=3_000)

In [3]:
df.shape

(3000, 4)

In [4]:
df.head()

,Title,Question,Id,Answer
0,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,535,"<p>I have very good experiences with <a href=""..."
1,Class views in Django,"<p><a href=""http://www.djangoproject.com/"">Dja...",742,<p>You can use the Django Generic Views. You c...
2,Python and MySQL,<p>I can get Python to work with Postgresql bu...,766,"<p>Take a pick at</p>\n\n<p><a href=""https://d..."
3,How do I use Python's itertools.groupby()?,<p>I haven't been able to find an understandab...,773,"<p>Another example:</p>\n\n<pre><code>for key,..."
4,Adding a Method to an Existing Object Instance,<p>I've read that it is possible to add a meth...,972,<p>I think that the above answers missed the k...


In [5]:
df.Answer.iloc[3]

"<p>Another example:</p>\n\n<pre><code>for key, igroup in itertools.groupby(xrange(12), lambda x: x // 5):\n    print key, list(igroup)\n</code></pre>\n\n<p>results in</p>\n\n<pre><code>0 [0, 1, 2, 3, 4]\n1 [5, 6, 7, 8, 9]\n2 [10, 11]\n</code></pre>\n\n<p>Note that igroup is an iterator (a sub-iterator as the documentation calls it).</p>\n\n<p>This is useful for chunking a generator:</p>\n\n<pre><code>def chunker(items, chunk_size):\n    '''Group items in chunks of chunk_size'''\n    for _key, group in itertools.groupby(enumerate(items), lambda x: x[0] // chunk_size):\n        yield (g[1] for g in group)\n\nwith open('file.txt') as fobj:\n    for chunk in chunker(fobj):\n        process(chunk)\n</code></pre>\n\n<p>Another example of groupby - when the keys are not sorted.  In the following example, items in xx are grouped by values in yy.  In this case, one set of zeros is output first, followed by a set of ones, followed again by a set of zeros.</p>\n\n<pre><code>xx = range(10)\nyy = 

In [19]:
ans = df.Answer.iloc[3]

In [20]:
ans

"<p>Another example:</p>\n\n<pre><code>for key, igroup in itertools.groupby(xrange(12), lambda x: x // 5):\n    print key, list(igroup)\n</code></pre>\n\n<p>results in</p>\n\n<pre><code>0 [0, 1, 2, 3, 4]\n1 [5, 6, 7, 8, 9]\n2 [10, 11]\n</code></pre>\n\n<p>Note that igroup is an iterator (a sub-iterator as the documentation calls it).</p>\n\n<p>This is useful for chunking a generator:</p>\n\n<pre><code>def chunker(items, chunk_size):\n    '''Group items in chunks of chunk_size'''\n    for _key, group in itertools.groupby(enumerate(items), lambda x: x[0] // chunk_size):\n        yield (g[1] for g in group)\n\nwith open('file.txt') as fobj:\n    for chunk in chunker(fobj):\n        process(chunk)\n</code></pre>\n\n<p>Another example of groupby - when the keys are not sorted.  In the following example, items in xx are grouped by values in yy.  In this case, one set of zeros is output first, followed by a set of ones, followed again by a set of zeros.</p>\n\n<pre><code>xx = range(10)\nyy = 

In [13]:
print(df.Answer.iloc[3])

<p>Another example:</p>

<pre><code>for key, igroup in itertools.groupby(xrange(12), lambda x: x // 5):
    print key, list(igroup)
</code></pre>

<p>results in</p>

<pre><code>0 [0, 1, 2, 3, 4]
1 [5, 6, 7, 8, 9]
2 [10, 11]
</code></pre>

<p>Note that igroup is an iterator (a sub-iterator as the documentation calls it).</p>

<p>This is useful for chunking a generator:</p>

<pre><code>def chunker(items, chunk_size):
    '''Group items in chunks of chunk_size'''
    for _key, group in itertools.groupby(enumerate(items), lambda x: x[0] // chunk_size):
        yield (g[1] for g in group)

with open('file.txt') as fobj:
    for chunk in chunker(fobj):
        process(chunk)
</code></pre>

<p>Another example of groupby - when the keys are not sorted.  In the following example, items in xx are grouped by values in yy.  In this case, one set of zeros is output first, followed by a set of ones, followed again by a set of zeros.</p>

<pre><code>xx = range(10)
yy = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0]


In [51]:
def clean_html(html_string):
    html_string = html.unescape(html_string)
    html_string = re.sub(r'<.+?>', '', html_string)
    html_string = html_string.replace('\r', '')
    return html_string.strip()

In [52]:
# format for search
qs = []
corpus = []
q2a = {}
for i in range(df.shape[0]):
    title = clean_html(df.Title.iloc[i])
    question = clean_html(df.Question.iloc[i])
    corp_entry = f"{title}\n{question}"
    corpus.append(corp_entry)

    question = title + ' ' + question
    #TODO: replace puncuation, =, <, >, " and brackets with spaces, will be better for tokinization
    #TODO: replace any more than 1 space with many spaces
    question = question.replace('\n', " ").replace('.', " ").replace('?', " ").replace('!', " ").replace('(', " ").replace(')', " ")
    question = question.replace('[', " ").replace(']', " ").replace('_', " ").replace('  ', ' ').strip().lower()
    qs.append(question.split(" "))

    answer = clean_html(df.Answer.iloc[i]).replace('  ', ' ').strip()
    q2a[corp_entry] = answer


In [53]:
bm25 = BM25Okapi(qs)

In [ ]:
def get_documents(query, corpus, n=1):
    tokenized_query = query.lower().split(' ')
    return bm25.get_top_n(tokenized_query, corpus, n=n)

In [ ]:
doc = get_documents("How do I drop duplicates from a pandas column?", corpus, 3)

In [ ]:
doc

["pandas drop_duplicates using comparison function\nIs it somehow possible to use pandas.drop_duplicates with a comparison operator which compares two objects in a particular column in order to identify duplicates? If not, what is the alternative?\n\nHere is an example where it could be used:\n\nI have a pandas DataFrame which has lists as values in a particular column and I would like to have duplicates removed based on column A\n\nimport pandas as pd\n\ndf = pd.DataFrame( {'A': [[1,2],[2,3],[1,2]]} )\nprint df\n\n\ngiving me\n\n        A\n0  [1, 2]\n1  [2, 3]\n2  [1, 2]\n\n\nUsing pandas.drop_duplicates\n\ndf.drop_duplicates( 'A' )\n\n\ngives me a TypeError\n\n[...]\nTypeError: type object argument after * must be a sequence, not itertools.imap\n\n\nHowever, my desired result is\n\n        A\n0  [1, 2]\n1  [2, 3]\n\n\nMy comparison function would be here:\n\ndef cmp(x,y):\n    return x==y\n\n\nBut in principle it could be something else, e.g.,\n\ndef cmp(x,y):\n    return x==y and le

In [ ]:
q2a[doc[0]]

'Option 1\n\ndf[~pd.DataFrame(df.A.values.tolist()).duplicated()]\n\n\n\n\nOption 2\n\ndf[~df.A.apply(pd.Series).duplicated()]'

In [27]:
doc = get_documents("How do I drop duplicates from a pandas column?", corpus, 3)

for d in doc:
    print(d)
    print('Answer:')
    print('\n', q2a[d])
    print("-"* 100)

pandas drop_duplicates using comparison function
Is it somehow possible to use pandas.drop_duplicates with a comparison operator which compares two objects in a particular column in order to identify duplicates? If not, what is the alternative?

Here is an example where it could be used:

I have a pandas DataFrame which has lists as values in a particular column and I would like to have duplicates removed based on column A

import pandas as pd

df = pd.DataFrame( {'A': [[1,2],[2,3],[1,2]]} )
print df


giving me

        A
0  [1, 2]
1  [2, 3]
2  [1, 2]


Using pandas.drop_duplicates

df.drop_duplicates( 'A' )


gives me a TypeError

[...]
TypeError: type object argument after * must be a sequence, not itertools.imap


However, my desired result is

        A
0  [1, 2]
1  [2, 3]


My comparison function would be here:

def cmp(x,y):
    return x==y


But in principle it could be something else, e.g.,

def cmp(x,y):
    return x==y and len(x)>1


How can I remove duplicates based on the 

In [32]:
q = """
Input to this function is a string represented multiple groups for nested parentheses separated by spaces.
    For each of the group, output the deepest level of nesting of parentheses.
    E.g. (()()) has maximum two levels of nesting while ((())) has three.
"""
doc = get_documents(q, corpus, 3)

for d in doc:
    print(d)
    print('Answer:')
    print('\n', q2a[d])
    print("-"* 100)

Easy way to determine a nesting level of nested tuples, in Python
Is there an easy way to determine the nesting level (without dependence on Python's recursion limit) of t (representing recombining binomial tree)?

t = (4, (3, 5, (2, 4, 6, (1, 3, 5, 7))))


Note that without a priori knowledge of depth of t, a routine may face recursion limit, which is set by sys.setrecursionlimit(n) and viewed by sys.getrecursionlimit(). Still, setting recursion limit very high before hand, may not be sufficient, thereby generating an error 

`RecursionError: maximum recursion depth exceeded while calling a Python object`. 


The following generates a larger (deeper) t: 

t = tuple(tuple(range(k)) for k in range(1,200))`


I guess these may work (have not worked out the details):


one can convert t to string and count number of opening brackets
If flattened tuple has size $N$, then the depth has size of the positive quadratic root of $n(n+1)/2=N$, that is $n=(-1+\sqrt(1+8N))/2$
iteratively peel (and 